# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/cities.csv", index_col = [0])
cities_df

,City,Country,Datetime,Latitude,Longitude,Max temperature (c),Humidity (%),Cloudiness (%),Wind Speed (mps)
0,ouesso,CG,1634715181,1.6136,16.0517,21.68,99,100,0.78
1,laguna,US,1634715183,38.4210,-121.4238,18.03,54,90,0.45
2,guerrero negro,MX,1634715184,27.9769,-114.0611,17.13,73,90,4.63
3,victoria,HK,1634715050,22.2855,114.1577,29.10,70,82,4.92
4,mataura,NZ,1634715186,-46.1927,168.8643,4.85,85,1,1.11
...,...,...,...,...,...,...,...,...,...
637,basoko,CD,1634715939,1.2391,23.6160,22.21,90,100,1.12
638,dolores,AR,1634716118,-36.3132,-57.6792,10.21,94,0,2.38
639,bajil,YE,1634716119,15.0584,43.2851,34.16,42,90,3.09
640,staryy nadym,RU,1634716121,65.6118,72.6842,1.95,76,100,7.26


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
import gmaps
gmaps.configure(api_key = g_key)

In [4]:
#Add locations to list and convert humidity to float for weighting
locations = cities_df[["Latitude", "Longitude"]]

humidity = cities_df["Humidity (%)"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
best_cities_df = cities_df.loc[(cities_df["Max temperature (c)"] > 20) 
                               & (cities_df["Max temperature (c)"] < 35) 
                               & (cities_df["Cloudiness (%)"] == 0)
                               & (cities_df["Humidity (%)"] < 30)].reset_index(drop = True)
best_cities_df

,City,Country,Datetime,Latitude,Longitude,Max temperature (c),Humidity (%),Cloudiness (%),Wind Speed (mps)
0,cape town,ZA,1634715018,-33.9258,18.4232,25.05,28,0,3.09
1,patiala,IN,1634715013,30.3267,76.4003,32.90,21,0,3.84
2,nawa,IN,1634715308,27.0167,75.0000,32.44,17,0,6.07
3,calvinia,ZA,1634715443,-31.4707,19.7760,24.90,14,0,8.23
4,buraydah,SA,1634715220,26.3260,43.9750,28.25,15,0,3.35
5,cradock,ZA,1634715728,-32.1642,25.6192,23.77,15,0,4.24
6,khash,IR,1634715769,28.2211,61.2158,29.57,13,0,4.46
7,aswan,EG,1634715597,24.0934,32.9070,27.61,28,0,6.17
8,shiraz,IR,1634715947,29.6036,52.5388,25.69,9,0,0.00
9,birjand,IR,1634716039,32.8663,59.2211,29.26,6,0,2.06


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
best_cities_df["Hotel Name"] = " "
best_cities_df

,City,Country,Datetime,Latitude,Longitude,Max temperature (c),Humidity (%),Cloudiness (%),Wind Speed (mps),Hotel Name
0,cape town,ZA,1634715018,-33.9258,18.4232,25.05,28,0,3.09,
1,patiala,IN,1634715013,30.3267,76.4003,32.90,21,0,3.84,
2,nawa,IN,1634715308,27.0167,75.0000,32.44,17,0,6.07,
3,calvinia,ZA,1634715443,-31.4707,19.7760,24.90,14,0,8.23,
4,buraydah,SA,1634715220,26.3260,43.9750,28.25,15,0,3.35,
5,cradock,ZA,1634715728,-32.1642,25.6192,23.77,15,0,4.24,
6,khash,IR,1634715769,28.2211,61.2158,29.57,13,0,4.46,
7,aswan,EG,1634715597,24.0934,32.9070,27.61,28,0,6.17,
8,shiraz,IR,1634715947,29.6036,52.5388,25.69,9,0,0.00,
9,birjand,IR,1634716039,32.8663,59.2211,29.26,6,0,2.06,


In [8]:
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"
    
for index, row in best_cities_df.iterrows():
    try:
        lat = row[3]
        lng = row[4]
        params = {
            "location": (f"{lat},{lng}"),
            "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
        }
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params = params).json()
        hotel_name = (response["results"][0]["name"])
        best_cities_df.loc[index,"Hotel Name"] = hotel_name
    except IndexError:
        print(f"Hotel could not be found near city: {row[0]}")
        pass

Hotel could not be found near city: nawa


In [9]:
#remove rows where no hotel could be found
hotel_df = best_cities_df.loc[best_cities_df["Hotel Name"] != " "].reset_index(drop=True)
hotel_df

,City,Country,Datetime,Latitude,Longitude,Max temperature (c),Humidity (%),Cloudiness (%),Wind Speed (mps),Hotel Name
0,cape town,ZA,1634715018,-33.9258,18.4232,25.05,28,0,3.09,The Bay Hotel
1,patiala,IN,1634715013,30.3267,76.4003,32.90,21,0,3.84,"The Baradari Palace - 19th Century, Patiala"
2,calvinia,ZA,1634715443,-31.4707,19.7760,24.90,14,0,8.23,Calvinia Hotel by Country Hotels
3,buraydah,SA,1634715220,26.3260,43.9750,28.25,15,0,3.35,Best Western Plus Buraidah
4,cradock,ZA,1634715728,-32.1642,25.6192,23.77,15,0,4.24,Die Tuishuise and Victoria Manor Cradock
5,khash,IR,1634715769,28.2211,61.2158,29.57,13,0,4.46,مسافر
6,aswan,EG,1634715597,24.0934,32.9070,27.61,28,0,6.17,Sofitel Legend Old Cataract Aswan
7,shiraz,IR,1634715947,29.6036,52.5388,25.69,9,0,0.00,Alan Boutique Hotel
8,birjand,IR,1634716039,32.8663,59.2211,29.26,6,0,2.06,هتل پرويز


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
#Set marker locations
marker_locations = hotel_df[["Latitude", "Longitude"]]
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity (%)"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

#Add marker layer
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)
fig.add_layer(markers)

#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius=1)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))